In [2]:
import pandas as pd
import numpy as np
# import os
from fastai.tabular.all import *
device = torch.device("cpu")


In [87]:

dtypes = {
	'insiderthreat': 'category',
	'vector': 'int32',
	'date': 'int32',
	'user': 'category',
	'source': 'category',
	'action': 'category'
}
df = pd.read_csv("scenario2-training-dataset-transformed-tf2.csv", dtype=dtypes)
decoder = { 'vector':
		{1: 'email',
		2: 'device',
		0: 'http'}
	}

df.replace(decoder, inplace=True)
print(df.info())
df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23231 entries, 0 to 23230
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   insiderthreat  23231 non-null  category
 1   vector         23231 non-null  object  
 2   date           23231 non-null  int32   
 3   user           23231 non-null  category
 4   source         23231 non-null  category
 5   action         22040 non-null  category
dtypes: category(4), int32(1), object(1)
memory usage: 554.6+ KB
None


,insiderthreat,vector,date,user,source,action
0,1,http,1280707200,CCH0959,PC-0588,http://linkedin.com/jobs/displayhome.html
1,1,http,1280707200,CCH0959,PC-0588,http://lockheedmartinjobs.com
2,1,http,1280707200,CCH0959,PC-0588,http://lockheedmartinjobs.com/searchall.aspx?sitesel=All
3,1,http,1280707200,CCH0959,PC-0588,http://indeed.com/Bronwyn_Bancroft/boomalli.html
4,1,http,1280793600,CCH0959,PC-0588,http://lockheedmartinjobs.com/searchhome.aspx


In [93]:
# df.groupby(['user', 'insiderthreat'])['source'].count().sort_values(ascending=False)
# df[(df.user == 'KSS1005') & (df.insiderthreat=='1')]
df[df.vector == 'device'].action.unique()

['Connect', 'Disconnect', 'R:;R:LBE0376', NaN, 'R:;R:BYO1846;R:89R53P6;R:49c9sY4;R:964CdV8', ..., 'R:;R:VCF1602', 'R:;R:WDT1634;R:35m7xf4;R:16RM723', 'R:;R:ZIE0741;R:99wjK21;R:35W1SS4;R:66D7387', 'R:;R:45ZpXY0;R:JPH1910', 'R:;R:93K57Z6;R:99M5KM4;R:CMP2946']
Length: 33
Categories (32, object): ['Connect', 'Disconnect', 'R:;R:LBE0376', 'R:;R:BYO1846;R:89R53P6;R:49c9sY4;R:964CdV8', ..., 'R:;R:WDT1634;R:35m7xf4;R:16RM723', 'R:;R:ZIE0741;R:99wjK21;R:35W1SS4;R:66D7387', 'R:;R:45ZpXY0;R:JPH1910', 'R:;R:93K57Z6;R:99M5KM4;R:CMP2946']

In [50]:
dep_var = 'insiderthreat'
cat_names = ['user', 'source', 'action', 'vector']
cat_names = ['action', 'vector']
cont_names = ['date']
procs = [Categorify, FillMissing, Normalize]

valid_size = 0.2
NUMBEROFSAMPLES = df.shape[0]

np.random.seed(42)
valid_idx = np.random.choice(range(NUMBEROFSAMPLES), int(NUMBEROFSAMPLES*valid_size))

dls = TabularDataLoaders.from_df(df, procs=procs, cat_names=cat_names, cont_names=cont_names,
				 y_names=dep_var, valid_idx=valid_idx, bs=64, device=device)
dls.show_batch()

,action,vector,date,insiderthreat
0,Connect,2,1.283990e+09,1
1,Connect,2,1.262784e+09,0
2,Connect,2,1.264804e+09,0
3,Connect,2,1.263554e+09,0
4,Connect,2,1.264782e+09,0
5,#na#,2,1.288742e+09,1
6,http://lockheedmartin.com/wbofrnepu1636367808.aspx,0,1.299024e+09,1
7,http://rice-part.bad.net,0,1.262870e+09,0
8,http://wikipedia.org,0,1.262871e+09,0
9,http://jobhuntersbible.com/WboUhagvat1258877042.aspx,0,1.301616e+09,1


In [51]:
learn = tabular_learner(dls, layers=[200,100], metrics=accuracy)


In [52]:
learn.fit_one_cycle(2)


epoch,train_loss,valid_loss,accuracy,time
0,0.002722,0.000627,1.000000,00:02
1,0.000596,0.000269,1.000000,00:02


In [53]:
learn.show_results()

,action,vector,date,insiderthreat,insiderthreat_pred
0,1416.0,1.0,-1.190519,0.0,0.0
1,378.0,1.0,-1.178696,0.0,0.0
2,410.0,1.0,0.289343,1.0,1.0
3,596.0,1.0,-1.196312,0.0,0.0
4,628.0,1.0,-0.050637,1.0,1.0
5,827.0,1.0,0.694703,1.0,1.0
6,32.0,3.0,0.949688,1.0,1.0
7,761.0,1.0,1.616572,1.0,1.0
8,410.0,1.0,1.021607,1.0,1.0


In [25]:
# Predict
row, clas, prob = learn.predict(df.iloc[15000])


In [27]:
row

    user  source  action  vector      date  insiderthreat
0  603.0   882.0    32.0     3.0 -1.163744            0.0

In [41]:
cols = ['id', 'date', 'user', 'pc', 'type', 'activity', 'content', 'threat']

In [48]:
def combinecsvs(rootpath, threat, list_of_files, outputpath):
	header = True
	mode = 'w'
	# Replace column names
	old_to_new_names = {'url': 'activity'}
	for file in list_of_files:
		chunks = pd.read_csv(rootpath + file, parse_dates=['date'], chunksize=100000)
		print("### Processing file", file, "###")
		for chunk_index, data in enumerate(chunks):
			# @preprocessing
			data['type'] = file.split(".")[0]
			data['threat'] = threat
			if 'content' not in data.columns:
				data['content'] = '-'

			data.rename(old_to_new_names, axis=1, inplace=True)

			data.to_csv(outputpath, header=header, mode=mode, 
				columns=cols, index=False)
			header=False
			mode='a'
			print('Processed Chunk #', chunk_index)

In [45]:

# TRUE POSITIVES
rootpath = "/home/haktrak/Public/InsiderThreatDataset/r3.1/"

list_of_files = ['device.csv', 'http.csv', 'logon.csv', 'email.csv', 'file.csv']
outputpath:str = "combined_truepositive.csv"
threat: bool = True
combinecsvs(rootpath, threat, list_of_files, outputpath)



### Processing file device.csv ###
Processed Chunk # 0
### Processing file http.csv ###
Processed Chunk # 0
Processed Chunk # 1
Processed Chunk # 2
Processed Chunk # 3
Processed Chunk # 4
Processed Chunk # 5
Processed Chunk # 6
Processed Chunk # 7
Processed Chunk # 8
Processed Chunk # 9
Processed Chunk # 10
Processed Chunk # 11
Processed Chunk # 12
Processed Chunk # 13
Processed Chunk # 14
Processed Chunk # 15
Processed Chunk # 16
Processed Chunk # 17
Processed Chunk # 18
Processed Chunk # 19
Processed Chunk # 20
Processed Chunk # 21
Processed Chunk # 22
Processed Chunk # 23
Processed Chunk # 24
Processed Chunk # 25
Processed Chunk # 26
Processed Chunk # 27
Processed Chunk # 28
Processed Chunk # 29
Processed Chunk # 30
Processed Chunk # 31
Processed Chunk # 32
Processed Chunk # 33
Processed Chunk # 34
### Processing file logon.csv ###
Processed Chunk # 0
Processed Chunk # 1
Processed Chunk # 2
Processed Chunk # 3
Processed Chunk # 4
Processed Chunk # 5
Processed Chunk # 6
Processed Chu

In [40]:
# Replace column names
old_to_new_names = {'url': 'activity'}
data.rename(old_to_new_names, axis=1, inplace=True)

# Interpret certain columns as categorical
cat_cols = ['user', 'pc', 'activity']
data[cat_cols] = data[cat_cols].astype('category')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        100000 non-null  object        
 1   date      100000 non-null  datetime64[ns]
 2   user      100000 non-null  category      
 3   pc        100000 non-null  category      
 4   activity  100000 non-null  category      
dtypes: category(3), datetime64[ns](1), object(1)
memory usage: 2.8+ MB


In [53]:
# TRUE POSITIVE

rootpath = "/home/haktrak/Public/InsiderThreatDataset/r3.1/"

# TRUE NEGATIVES (which are REALLY BIG)
list_of_files = ['device.csv', 'http.csv', 'logon.csv', 'file.csv', 'email.csv']
data = pd.read_csv(rootpath + list_of_files[1], parse_dates=['date'], chunksize=10)

instance = next(data)
print(instance.info())
instance.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   id       10 non-null     object        
 1   date     10 non-null     datetime64[ns]
 2   user     10 non-null     object        
 3   pc       10 non-null     object        
 4   url      10 non-null     object        
 5   content  10 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 608.0+ bytes
None


,id,date,user,pc,url,content
0,{U0G4-P5TW04EO-9366IUQF},2010-01-02 06:36:20,EBH0519,PC-9573,http://usbank.com/1930_FIFA_World_Cup/patenaud...,neros printing pastiche four topiary body pink...
1,{G0R5-U4RX40DR-3710WPUY},2010-01-02 06:39:22,EBH0519,PC-9573,http://forbes.com/Chorioactis/urnula.html,balhae fubing guotu namsaeng geng chuzhou ying...
2,{G0Q6-F5CV96RX-0355APNP},2010-01-02 06:39:53,EBH0519,PC-9573,http://coupons.com/BeauchampSharp_Tragedy/darb...,ziyi hazelnuts 34850 stat_area1 namsaeng kucha...
3,{J8X0-C9ZO08VS-9370QIFO},2010-01-02 06:40:56,EBH0519,PC-9573,http://youtube.com/Amanita_ocreata/velosa.html,flambard anselm anselm precentor thurgot visit...
4,{E0J4-D6WM72LS-9186JPKE},2010-01-02 06:47:23,EBH0519,PC-9573,http://twitter.com/Cannon/m198.html,maternus henrici anselm walkelin maternus thur...


In [57]:
instance.iloc[4]['content']

'maternus henrici anselm walkelin maternus thurgot harcourt gerard histamine promyelocytic ceplene leukemic anthracycline chloroma npm1 monoblastic chloroma schemata neutropenia apl virchow myeloproliferative cytochemical proleukin saccadic chorea genetically increasingly institutionalization models akinetic hypersexuality abilities psychomotor chorea subcortical chorea unsteady unexploded chosin muzzle 1288 jalut 694 1280 vicomte m198 chosin 1643 breitenfeld m198'

In [ ]:


#Split the dataframe into train, validation, and test
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')
#Create an input pipeline using tf.data
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
	dataframe = dataframe.copy()
	labels = dataframe.pop('insiderthreat')
	ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
	if shuffle:
		ds = ds.shuffle(buffer_size=len(dataframe))
	ds = ds.batch(batch_size)
	return ds
#choose columns needed for calculations (features)
feature_columns = []
for header in ["vector", "date", "user", "source", "action"]:
    feature_columns.append(feature_column.numeric_column(header))
#create feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
#set batch size pipeline
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)
#create compile and train model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)